# Домашнее задание. Обучение языковой модели с помощью LSTM (10 баллов)

Это домашнее задание проходит в формате peer-review. Это означает, что его будут проверять ваши однокурсники. Поэтому пишите разборчивый код, добавляйте комментарии и пишите выводы после проделанной работы. 

В этом задании Вам предстоит обучить языковую модель с помощью рекуррентной нейронной сети. В отличие от семинарского занятия, Вам необходимо будет работать с отдельными словами, а не буквами.


Установим модуль ```datasets```, чтобы нам проще было работать с данными.

In [ ]:
!pip install datasets

In [ ]:
# Импорт необходимых библиотек
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import nltk
import string

from collections import Counter
from typing import List

import seaborn
seaborn.set(palette='summer')

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Воспользуемся датасетом imdb. В нем хранятся отзывы о фильмах с сайта imdb. Загрузим данные с помощью функции ```load_dataset```

In [ ]:
# Загрузим датасет
dataset = load_dataset('imdb')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

### Препроцессинг данных и создание словаря (1 балл)

In [ ]:
dataset['train']['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [ ]:
sentences = []
word_threshold = 32

# Получить отдельные предложения и поместить их в sentences
for sentence in tqdm(dataset['train']['text']):
    sentences.extend([x.lower() for x in sent_tokenize(sentence) if len(x.split()) < word_threshold])

  0%|          | 0/25000 [00:00<?, ?it/s]

In [ ]:
print("Всего предложений:", len(sentences))

Всего предложений: 219930


In [ ]:
sentences[0]

'i rented i am curious-yellow from my video store because of all the controversy that surrounded it when it was first released in 1967.'

Посчитаем для каждого слова его встречаемость.

In [ ]:
words = Counter()

# Расчет встречаемости слов

for sentence in sentences:
    words.update(word_tokenize(sentence))

Добавим в словарь ```vocab_size``` самых встречающихся слов.

In [ ]:
len(words)

77795

In [ ]:
vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])
vocab_size = 40000

# Наполнение словаря
most_common_words = words.most_common(vocab_size)
vocab.update([word for word, _ in most_common_words])

In [ ]:
assert '<unk>' in vocab
assert '<bos>' in vocab 
assert '<eos>' in vocab 
assert '<pad>' in vocab
assert len(vocab) == vocab_size + 4

In [ ]:
print("Всего слов в словаре:", len(vocab))

Всего слов в словаре: 40004


### Подготовка датасета (1 балл)

In [ ]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [ ]:
class WordDataset:
    def __init__(self, sentences):
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        tokenized_sentence = [self.bos_id]
        tokenized_sentence += [word2ind.get(char, self.unk_id) for char in self.data[idx].split(' ')]
        tokenized_sentence += [self.eos_id]
        return tokenized_sentence

    def __len__(self) -> int:
        return len(self.data)

In [ ]:
def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id=word2ind['<pad>']) -> torch.Tensor:
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)
    
    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:,:-1],
        'target_ids': sequences[:,1:]
    }

    return new_batch

In [ ]:
train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2, random_state=69)
eval_sentences, test_sentences = train_test_split(sentences, test_size=0.5, random_state=69)

train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)
test_dataset = WordDataset(test_sentences)

batch_size = 40

train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

## Обучение и архитектура модели

Вам необходимо на практике проверить, что влияет на качество языковых моделей. В этом задании нужно провести серию экспериментов с различными вариантами языковых моделей и сравнить различия в конечной перплексии на тестовом множестве. 

Возмоэные идеи для экспериментов:

* Различные RNN-блоки, например, LSTM или GRU. Также можно добавить сразу несколько RNN блоков друг над другом с помощью аргумента num_layers. Вам поможет официальная документация [здесь](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
* Различные размеры скрытого состояния. Различное количество линейных слоев после RNN-блока. Различные функции активации. 
* Добавление нормализаций в виде Dropout, BatchNorm или LayerNorm
* Различные аргументы для оптимизации, например, подбор оптимального learning rate или тип алгоритма оптимизации SGD, Adam, RMSProp и другие
* Любые другие идеи и подходы

После проведения экспериментов необходимо составить таблицу результатов, в которой описан каждый эксперимент и посчитана перплексия на тестовом множестве. 

Учтите, что эксперименты, которые различаются, например, только размером скрытого состояния или количеством линейных слоев считаются, как один эксперимент.

Успехов!

### Функция evaluate (1 балл)

In [ ]:
def evaluate(model, criterion, dataloader) -> float:
    model.eval()
    perplexity = []
    with torch.no_grad():
        for batch in dataloader:
            logits = model(batch['input_ids']).flatten(start_dim = 0, end_dim=1)
            loss = criterion(logits, batch['target_ids'].flatten())
            perplexity.append(torch.exp(loss).item())
    
    perplexity = sum(perplexity) / len(perplexity)

    return perplexity

In [ ]:
def generate_sequence(model, starting_seq:str, max_seq_len:int=128):
    device = 'cpu'
    model = model.to(device)
    input_ids = [word2ind['<bos>']] + [
      word2ind.get(char, word2ind['<unk>']) for char in  starting_seq]
    input_ids = torch.LongTensor(input_ids).to(device)
    model.eval()
    with torch.no_grad():
        for i in range(max_seq_len):
            next_char_distribution = model(input_ids)[-1]
            next_char = next_char_distribution.squeeze().argmax()
            input_ids = torch.cat([input_ids, next_char.unsqueeze(0)])

            if next_char.item() == word2ind['<eos>']:
                break

    words = ''.join([ind2word[idx.item()] for idx in input_ids]) 
    
    return words

### Train loop (1 балл)

In [ ]:
def train_model(model, criterion, optimizer, train_dataloader, eval_dataloader, num_epochs):

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        num_batches = len(train_dataloader)

        for batch in tqdm(train_dataloader, desc=f'Training epoch {epoch}'):
            optimizer.zero_grad()

            input_ids = batch['input_ids']
            target_ids = batch['target_ids']

            logits = model(input_ids).flatten(start_dim=0, end_dim=1)
            loss = criterion(logits, target_ids.flatten())
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / num_batches
        # рассчет perplexity на тестовой выборки на данную эпоху
        perplexity = evaluate(model, criterion, test_dataloader)

        print(f"Epoch {epoch+1}/{num_epochs} - Avg. Loss: {avg_loss:.4f} - Perplexity: {perplexity:.4f}")

### Первый эксперимент (2 балла)
 

In [ ]:
class LanguageModel_1(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_layers):
        super(LanguageModel_1, self).__init__()       
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.nonlin = nn.Tanh()
        self.dropout = nn.Dropout(.1)
    
    def forward(self, input_batch):
        embedded = self.embedding(input_batch)
        output, _ = self.lstm(embedded)
        output = self.dropout(self.linear(self.nonlin(output)))
        projection = self.projection(self.nonlin(output))
        
        return projection

In [ ]:
model = LanguageModel_1(vocab_size=vocab_size, hidden_dim=128, num_layers=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
num_epochs = 5

In [ ]:
train_model(model, criterion, optimizer, train_dataloader, eval_dataloader, num_epochs)

Training epoch 0:   0%|          | 0/2750 [00:00<?, ?it/s]

Epoch 1/5 - Avg. Loss: 2.3916 - Perplexity: 5.4273


Training epoch 1:   0%|          | 0/2750 [00:00<?, ?it/s]

Epoch 2/5 - Avg. Loss: 1.6001 - Perplexity: 4.4818


Training epoch 2:   0%|          | 0/2750 [00:00<?, ?it/s]

Epoch 3/5 - Avg. Loss: 1.4830 - Perplexity: 4.1873


Training epoch 3:   0%|          | 0/2750 [00:00<?, ?it/s]

Epoch 4/5 - Avg. Loss: 1.4261 - Perplexity: 4.0169


Training epoch 4:   0%|          | 0/2750 [00:00<?, ?it/s]

Epoch 5/5 - Avg. Loss: 1.3892 - Perplexity: 3.8951


In [ ]:
generate_sequence(model, starting_seq=' my favorite place at')

'<bos><unk>my<unk>favorite<unk>place<unk>at<unk>the<unk>film<unk>is<unk>a<unk>story<unk>and<unk>the<unk>story<unk>is<unk>a<unk>strange<unk>of<unk>the<unk>film<unk>and<unk>the<unk>story<unk>is<unk>a<unk>strange<unk>of<unk>the<unk>film<unk>and<unk>the<unk>story<unk>is<unk>a<unk>strange<unk>of'

### Второй эксперимент (2 балла)

Во втором эксперименте увеличим hidden_dim c 128 до 256, кол-во слоев LSTM до 4 и в конце поменяем оптимизатор на SGD.

In [ ]:
class LanguageModel_2(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_layers):
        super(LanguageModel_2, self).__init__()       
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.nonlin = nn.Tanh()
        self.dropout = nn.Dropout(.1)
    
    def forward(self, input_batch):
        embedded = self.embedding(input_batch)
        output, _ = self.lstm(embedded)
        output = self.dropout(self.linear(self.nonlin(output)))
        projection = self.projection(self.nonlin(output))
        
        return projection

In [ ]:
model = LanguageModel_2(vocab_size=vocab_size, hidden_dim=256, num_layers=4).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
num_epochs = 5

In [ ]:
train_model(model, criterion, optimizer, train_dataloader, eval_dataloader, num_epochs)

Training epoch 0:   0%|          | 0/4399 [00:00<?, ?it/s]

Epoch 1/5 - Avg. Loss: 10.1599 - Perplexity: 9475.5176


Training epoch 1:   0%|          | 0/4399 [00:00<?, ?it/s]

Epoch 2/5 - Avg. Loss: 5.2492 - Perplexity: 35.0019


Training epoch 2:   0%|          | 0/4399 [00:00<?, ?it/s]

Epoch 3/5 - Avg. Loss: 3.3597 - Perplexity: 25.5456


Training epoch 3:   0%|          | 0/4399 [00:00<?, ?it/s]

Epoch 4/5 - Avg. Loss: 3.2232 - Perplexity: 24.3427


Training epoch 4:   0%|          | 0/4399 [00:00<?, ?it/s]

Epoch 5/5 - Avg. Loss: 3.1842 - Perplexity: 23.6976


In [ ]:
generate_sequence(model, starting_seq=' my favorite place at')

'<bos><unk>my<unk>favorite<unk>place<unk>at<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>'

# Отчет (2 балла)

В этой тетраде мы разобрали задачу языкового моделирования нейросетевыми подходами на примере imdb-review.

Эксперимент №1.

В качестве модели взяли пример с семинара, но чуть с изменениями мы построили 2 слоя LSTM с функ. акт. nn.Tanh для следующего линейного слоя , затем Dropout с 0.1 и проекция
Обучали данную модель 5 эпох с помощью оптимизатора Adam, с размером скрытых слоев сети 256.

Perplexity была равна  3.8951

Эксперимент №2.

Архитектуру сети мы оставили без изменений, но структура поменялась: Изменили Dropout на 0.3, hidden_dim c 128 до 256, кол-во слоев LSTM до 4 и в конце поменяли оптимизатор на SGD.

Perplexity была равна  23.6976


Так как лосс и перплексия падали в обеих случаях, то мы могли увеличить кол-во эпох до сходимости модели, также увеличить размер словаря изменением вхождением предложений в обучающую выборку и как итог изменить архитектуру сети(усложнить) и поиграться с гиперпараметрами.

P.s. я попытался скопировать код для генерации текста с семинара , но он работает не до конца корректно поэтому не серчайте.